# Audio Reconstruction - auto encoders

Avant de passer à la régression de paramètres de synthé, on va d'abord entraîner un AE à reconstruire les (mel-spectrogrammes) des sons des presets du DX7.

Choix du AE :
* Encodeur/décodeur à CNN sur spectrogramme (FlowSynth, IRCAM 2019)
    * WAE, qui d'après (Esling et al. 2020) donne au final les meilleures régressions de paramètres de synthé ?
    * Divers VAE avec flow, qui d'après (Esling et al. 2020) donnent la meilleure reconstruction audio ? (avec metadata....)
* WaveNet
    * Temporal encoder à convolution sur raw audio
        * Comparer le *baseline CNN encoder* avec le *temporal encoder* sur spectrogrammes ? Même si Google Brain/DeepMind restent sur du CNN classique 
    * Décodeur WaveNet : beaucoup trop lourd

Questions en suspens :
* Comment implémenter la régularisation MMD du WAE ?
    * Avec "Inverse Multiquadratics" kernel, cf. WAE 2018
* VAE avec normalizing flow : est-ce qu'on fait la régularisation au début ou à la fin du flow ?
    * Pour WAE : même problème...
* Intégrer le pitch qqpart va sans doute beaucoup aider
    * Pour comprendre tous les params, plusieurs notes//plusieurs pitch sont probablement nécessaires
* Problème général des synthés qui peuvent générer le même son de plusieurs manières différentes

Dimension de l'espace latent z :
* FlowSynth (Esling et al. 2020) prend $d_z = d_v$ avec $v$ le vecteur des paramètres. En effet, ils ont voulu un flow inversible entre z et v
    * $+$ : traduction facile d'un preset en vecteur latent z
    * $+$ : de chaque côté de l'interpolation, on a *exactement* le preset d'origine...
    * $+$ : chaque réel $z_i$ peut se rapprocher d'une dimension perceptuelle de haut-niveau
    * $-$ : limite de taille de l'espace latent
    * $-$ : autant de "macro-contrôles" que de params de synthé (solution possible : interp miem...)
    * $-$ : MSE assez forte sur valeurs de paramètres (flow inversible pas si puissant qu'un NN)
* WaveNet AE (Engel et al. 2017) encode un z à 2 dimensions : 16 dimensions par timestep, 125 timesteps de 512 samples (32 ms à 16kHz)
    * Un latent z comme ça permet sans doute de bien représenter attaque/sustain/decay, LFO, etc ? Et donc d'obtenir une bonne reconstruction

## Recherche pour publication début 2021, spécifique sur l'interpolation de paramètres

Se focaliser sur l'entraînement et la généralisation à tout synthé - pour interpolation de presets.

* *Structure de base : WAE avec enc/dec à CNN sur spectrogramme ?*
    * *(on pourrait aussi essayer le temporal encoder wavenet)*
* Extension du taf de Esling et al. 2019/2020 
    * Résultats très encourageants, mais il y a clairement *room for improvement*
        * Nb de params du synthé : limités
        * À tester sur d'autres synthés, comme le DX7 (synthèse + complexe mais pas d'effets intégrés)
    * Dans l'objectif de l'interp. de preset
        * **Étude de continuité sonore de l'interpolation ?**
        * Pour n'importe quel synth, avec peu de presets, zéro meta-tag
        * Pure unsupervised learning
        * **Étude applicative, en explorant différents synthés et différents spectrogrammes ?**
            * *Avantage de faire ça : peu importe que ça ne marche pas bien avec un synthé ou l'autre...*
    * Leurs résultats comparaient déjà les 2 approches intéressantes
        * **Flow inversible z <-> v, qui est la solution idéale pour l'interpolation**
        * WAE avec MLP z -> v ??
            * MLP régression sauvage
            * MLP avec modélisation fine (sortie cat. vs sorties gaussiennes)
        * **Tester d'autres NN inversibles ?** Sans forcément modéliser toutes les maths...?
            * Le domaine des INN, c'est un peu le domaine des flows...
    * Autre expé avec le DX7, avec comme objectifs
        * Confirmer ou infirmer résultats, pour un autre synthé
            * En utilisant *tous* les paramètres du synthé, pas un subset choisi (max 64 sur 200+ dans Diva)
            * **Problème du pitch/vel qui influe --> entraîner un NN AVEC PITCH ET VEL DANS LE VAE ?? Il faudrait plusieurs spectrogrammes pour 1 seule estimation**
        * Essayer d'améliorer la modélisation de l'audio avec NNs plus gros
        * Tester **data augmentation** en modifiant légèrement certaines valeurs de paramètres avant de générer en live un spectrogramme
            * Essayer un synthé avec beaucoup moins de preset. En pratique, on a rarement 10000 ou 30000 presets (et qualité discutable) comme avec diva et/ou le DX7.
        * Discuter des avantages/inconvénients du MLP z->v vs. flow z<->v

# -